# Installing requirements

In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\sarac\\OneDrive - Università degli Studi di Milano\\[erasmus]\\[COURSES]\\[DB]\\assignments\\assignment2\\GITHUB\\Very_large_distributed_data_volumes_exercise2\\code'

In [3]:
#path = os.getcwd()
#os.chdir(path+"/code")

In [4]:
#os.getcwd()

In [5]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\sarac\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
os.chdir("..")
os.getcwd()

'c:\\Users\\sarac\\OneDrive - Università degli Studi di Milano\\[erasmus]\\[COURSES]\\[DB]\\assignments\\assignment2\\GITHUB\\Very_large_distributed_data_volumes_exercise2'

# Imports

In [4]:
from tabulate import tabulate
import mysql.connector as mysql

# Class DBConnector

In [5]:
class DbConnector:
    """
    Connects to the MySQL server on the Ubuntu virtual machine.
    Connector needs HOST, DATABASE, USER and PASSWORD to connect,
    while PORT is optional and should be 3306.

    Example:
    HOST = "tdt4225-00.idi.ntnu.no" // Your server IP address/domain name
    DATABASE = "testdb" // Database name, if you just want to connect to MySQL server, leave it empty
    USER = "testuser" // This is the user you created and added privileges for
    PASSWORD = "test123" // The password you set for said user
    """

    def __init__(self,
                 HOST="tdt4225-19.idi.ntnu.no",
                 DATABASE="test_19",
                 USER="team19",
                 PASSWORD="team19*"):
        # Connect to the database
        try:
            self.db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD, port=3306)
        except Exception as e:
            print("ERROR: Failed to connect to db:", e)

        # Get the db cursor
        self.cursor = self.db_connection.cursor()

        print("Connected to:", self.db_connection.get_server_info())
        # get database information
        self.cursor.execute("select database();")
        database_name = self.cursor.fetchone()
        print("You are connected to the database:", database_name)
        print("-----------------------------------------------\n")

    def close_connection(self):
        # close the cursor
        self.cursor.close()
        # close the DB connection
        self.db_connection.close()
        print("\n-----------------------------------------------")
        print("Connection to %s is closed" % self.db_connection.get_server_info())


# Example program

In [85]:
class ExampleProgram:

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

    def create_table(self, table_name):
        query = """CREATE TABLE IF NOT EXISTS %s (
                   id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
                   name VARCHAR(30))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % table_name)
        self.db_connection.commit()

    def insert_data(self, table_name):
        names = ['Bobby', 'Mc', 'McSmack', 'Board']
        for name in names:
            # Take note that the name is wrapped in '' --> '%s' because it is a string,
            # while an int would be %s etc
            query = "INSERT INTO %s (name) VALUES ('%s')"
            self.cursor.execute(query % (table_name, name))
        self.db_connection.commit()

    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


def main():
    program = None
    try:
        program = ExampleProgram()
        program.create_table(table_name="Person")
        program.insert_data(table_name="Person")
        _ = program.fetch_data(table_name="Person")
        program.drop_table(table_name="Person")
        # Check that the table is dropped
        program.show_tables()
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()


In [86]:
if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

Data from table Person, raw format:
[(1, 'Bobby'), (2, 'Mc'), (3, 'McSmack'), (4, 'Board')]
Data from table Person, tabulated:
  id  name
----  -------
   1  Bobby
   2  Mc
   3  McSmack
   4  Board
Dropping table Person...
Tables_in_test_19
-------------------
Activity_s
Activity_v
TrackPoint_s
TrackPoint_v
User_s
User_v

-----------------------------------------------
Connection to None is closed


# Task 1

### Operations on the tables

In [19]:
class Task1:

    # we use constant for defining the tables
    USER = 'User'
    ACTIVITY = 'Activity'
    TRACK_POINT = 'TrackPoint'

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

# --------------- CREATION OF THE TABLES ---------------

    def create_table_user(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    userID VARCHAR(30) NOT NULL PRIMARY KEY,
                    has_labels BOOLEAN)
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.USER)
        self.db_connection.commit()
    

    def create_table_activity(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    activityID int AUTO_INCREMENT NOT NULL,
                    userID varchar(30) NOT NULL,	
                    transportation_mode varchar(30),
                    start_date_time datetime NOT NULL,
                    end_date_time datetime NOT NULL,
                    PRIMARY KEY (activityID),
                    FOREIGN KEY (userID) REFERENCES User(userID)
                )"""
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.ACTIVITY)
        self.db_connection.commit()

        query_constr = """ ALTER TABLE %s
                    ADD CONSTRAINT check_start_end_dates
                    CHECK (start_date_time < end_date_time)
                """

        self.cursor.execute(query_constr % self.ACTIVITY)
        self.db_connection.commit()
    

    def create_table_trackpoint(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    trackPointID INT AUTO_INCREMENT NOT NULL,
                    activityID int NOT NULL,
	                lat double NOT NULL,
	                lon double NOT NULL,
	                altitude int NOT NULL,
	                date_days double NOT NULL,
	                date_time datetime NOT NULL,
	                PRIMARY KEY (trackPointID),
	                FOREIGN KEY (activityID) REFERENCES Activity(activityID))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.TRACK_POINT)
        self.db_connection.commit()

# --------------- INSERT OF DATA ---------------

# FIXME

    def insert_data_user(self):
        # we go to the Data directory
        path = os.getcwd()
        os.chdir(path+"\\dataset\\dataset\\Data")
        

        # we get the list of all the diferent directory names (users ids) and we sort the list
        usersID =  os.listdir()
        usersID.sort()

        # we go back to the dataset directory and we read the labeled_ids.txt file
        os.chdir("..")
        with open('labeled_ids.txt') as f:
            labeled_ids = f.readlines()
        f.close()
        os.chdir("..")
        os.chdir("..")
        
        # # we delete the \n in each string
        for i in range (0, len(labeled_ids)):
            labeled_ids[i] = labeled_ids[i].strip()

        # we check if each user has a label or not and we save the info in a list
        # the indexes of has_labels and id lists are correponding
        has_labels = []
        for i in usersID : 
            if i in labeled_ids : 
                has_labels.append(True)
            else:
                has_labels.append(False)

        # we execute the query to insert the data
        for i in range(len(usersID)):

            # NOTE: if we want to insert a string -> '%s'
            query = "INSERT INTO %s (userID, has_labels) VALUES ('%s', %s)"
            values = (self.USER, usersID[i], has_labels[i])
            self.cursor.execute(query % (values))

        self.db_connection.commit()

# --------------- FETCH DATA ---------------

    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows

# --------------- DROP TABLE ---------------

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

# --------------- SHOW TABLES ---------------

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


In [22]:
def main():
    program = None

    try:
        program = Task1() 

        # --- TABLE CREATION ---
        # program.create_table_user()
        # program.create_table_activity()
        # program.create_table_trackpoint()

        # --- INSERTION ---
        #program.insert_data_user()

        # --- DROP TABLE ---
        # program.drop_table(program.TRACK_POINT)
        # program.drop_table(program.ACTIVITY)
        # program.drop_table(program.USER)

        # --- FETCH ---
        program.fetch_data(program.USER)
        
        # --- SHOW TABLES ---
        #program.show_tables()
        
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()

if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

Data from table User, raw format:
[('000', 0), ('001', 0), ('002', 0), ('003', 0), ('004', 0), ('005', 0), ('006', 0), ('007', 0), ('008', 0), ('009', 0), ('010', 1), ('011', 0), ('012', 0), ('013', 0), ('014', 0), ('015', 0), ('016', 0), ('017', 0), ('018', 0), ('019', 0), ('020', 1), ('021', 1), ('022', 0), ('023', 0), ('024', 0), ('025', 0), ('026', 0), ('027', 0), ('028', 0), ('029', 0), ('030', 0), ('031', 0), ('032', 0), ('033', 0), ('034', 0), ('035', 0), ('036', 0), ('037', 0), ('038', 0), ('039', 0), ('040', 0), ('041', 0), ('042', 0), ('043', 0), ('044', 0), ('045', 0), ('046', 0), ('047', 0), ('048', 0), ('049', 0), ('050', 0), ('051', 0), ('052', 1), ('053', 1), ('054', 0), ('055', 0), ('056', 1), ('057', 0), ('058', 1), ('059', 1), ('060', 1), ('061', 0), ('062', 1), ('063', 0), ('064', 1), ('065', 1), ('066', 0), ('067', 1), ('068', 1), ('

In [108]:
os.chdir("..")
os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

In [109]:
path = os.getcwd()
os.chdir(path+"/dataset/dataset/Data")

In [133]:
from datetime import datetime
user_id = []
transportation_mode = []
start_date_time = []
end_date_time = []

for dirpath, dirnames, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith('.txt'):
            with open(os.path.join(dirpath, filename)) as f:
                lines = f.readlines()
                # skip the header
                lines = lines[1:]
                for line in lines:

                    data = line.split()

                    last_directory_name = os.path.basename(dirpath)
                    user_id.append(last_directory_name)

                    transportation_mode.append(data[4])

                    start_date = data[0]
                    start_time = data[1]
                    end_date = data[2]
                    end_time = data[3]
                    start_datetime_str = start_date + " " + start_time
                    end_datetime_str = end_date + " " + end_time
                    combined_start_datetime = datetime.strptime(start_datetime_str, "%Y/%m/%d %H:%M:%S")
                    combined_end_datetime = datetime.strptime(end_datetime_str, "%Y/%m/%d %H:%M:%S")

                    start_date_time.append(combined_start_datetime)
                    end_date_time.append(combined_end_datetime)
            f.close()

In [136]:
len(user_id)

14718

In [137]:
len(transportation_mode)

14718

In [138]:
len(start_date_time)

14718